In [8]:
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import pandas as pd
import random
import seaborn as sns
import warnings
import itertools
import sklearn.neighbors
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import sklearn.linear_model
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

plt.style.use('ggplot')

In [9]:
train = pd.read_csv('train.csv').set_index('PassengerId')
test = pd.read_csv('test.csv').set_index('PassengerId')
test['Survived'] = 0

train_and_test = pd.concat([train, test])

train_and_test['Rank'] = train_and_test['Cabin'].str[0]
train_and_test = pd.get_dummies(train_and_test, columns=['Sex', 'Pclass', 'Rank', 'Embarked'])

def normalize(column):
    train_and_test[column] = (train_and_test[column] - train_and_test[column].min()) / (train_and_test[column].max() - train_and_test[column].min())
    
train_and_test['LastName'] = train_and_test['Name'].apply(lambda x: x.split(',')[0])
survived_families = train_and_test[train_and_test['Survived'] != -1].groupby('LastName')
survived_families = dict((survived_families['Survived'].sum() - 1) / survived_families['Survived'].count())
train_and_test['SurvivedFamily'] = train_and_test['LastName'].apply(lambda name: survived_families.get(name, 0))
normalize('SurvivedFamily')

train_and_test['Age'] = train_and_test['Age'].fillna(30)
normalize('Age')
normalize('SibSp')
normalize('Parch')
normalize('Fare')
train_and_test['Fare'] = train_and_test['Fare'].fillna(0)

train_and_test = train_and_test.drop(['Name', 'Ticket', 'Cabin', 'LastName'], axis=1)

train_and_test['Survived'] = train_and_test.pop('Survived')

data = train_and_test[~train_and_test.index.isin(test.index)]
test = train_and_test[~train_and_test.index.isin(data.index)]

data = data.reset_index()
test = test.reset_index()

X_train, X_test = train_test_split(data.to_numpy())

data.set_index('PassengerId').to_csv('tmd_data.csv')

In [10]:
model = sklearn.linear_model.LogisticRegression()
param_grid = {}
lin_gscv = GridSearchCV(model, param_grid, cv=5)
lin_gscv.fit(X[:, 1:-1], X[:, -1])

print("%s: %s score: %s" %  ("LogReg", str(lin_gscv.best_params_), str(lin_gscv.best_score_)))


LogReg: {} score: 0.8967359236708304


In [4]:
model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=2)
param_grid = {
    "n_estimators": range(10, 50, 5),
    "max_features": range(4, 10, 100),
    "max_depth": range(5, 10)
}
rfc_gscv = GridSearchCV(model, param_grid, cv=10, n_jobs=-1)
rfc_gscv.fit(X[:, 1:-1], X[:, -1])
print("%s: %s score: %s" %  ("Forest", str(rfc_gscv.best_params_), str(rfc_gscv.best_score_)))

Forest: {'max_depth': 8, 'max_features': 4, 'n_estimators': 35} score: 0.9427840199750313


In [5]:
model = MLPClassifier(random_state=1, activation='relu')
param_grid = {
    'max_iter': np.arange(10, 50, step=10),
    'solver': ['lbfgs', 'adam'],
#    'hidden_layer_sizes': list(range(4, 10)) + list(itertools.product(range(4, 12), range(1, 8)))
    'hidden_layer_sizes': list(itertools.product(range(18, 24), range(4, 8)))
}
mlp_gscv = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)
mlp_gscv.fit(X[:, 1:-1], X[:, -1])
print("%s: %s score: %s" %  ('MLP', str(mlp_gscv.best_params_), str(mlp_gscv.best_score_)))
# 0.8271

MLP: {'hidden_layer_sizes': (21, 6), 'max_iter': 40, 'solver': 'lbfgs'} score: 0.9315422760655327


In [6]:
def predict(model):
    test_data = test.to_numpy()
    predicted = mlp_gscv.predict(test_data[:, 1:-1])
    predicted = np.c_[test_data[:, 0], predicted]

    predicted = pd.DataFrame(data=predicted, columns=['PassengerId', 'Survived'])

    predicted['PassengerId'] = predicted['PassengerId'].astype(int)
    predicted['Survived'] = predicted['Survived'].astype(int)
    return predicted.set_index('PassengerId').sort_values(by=['PassengerId'])

predict_lin = predict(lin_gscv)
predict_rfc = predict(rfc_gscv)
predict_mpl = predict(mlp_gscv)

predict_mpl.to_csv('my_submission.csv')

predict_lin['From'] = 'lin'
predict_rfc['From'] = 'rfc'
predict_mpl['From'] = 'mlp'

pd.concat([predict_lin, predict_rfc, predict_mpl]).sort_values(by=['PassengerId']).drop_duplicates('Survived')

,Survived,From
PassengerId,,
892,1,lin
893,0,lin
